In [1]:
import sys
sys.path.append('C:/Users/user/Documents/GitHub/KAIRI_MCD/monte-carlo-diffusion') 

%pip install C:/Users/user/Documents/GitHub/KAIRI_MCD/monte-carlo-diffusion 

%pip install --upgrade pip


Processing c:\users\user\documents\github\kairi_mcd\monte-carlo-diffusionNote: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.



  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     --------------------------------------- 51.9/51.9 MB 36.4 MB/s eta 0:00:00
     ---------------------------------------- 178.9/178.9 KB ? eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 43.5 MB/s eta 0:00:00
     ---------------------------------------- 55.8/55.8 KB ? eta 0:00:00
     ---------------------------------------- 356.1/356.1 KB ? eta 0:00:00
     ---------------------------------------- 209.4/209.4 KB ? eta 0:00:00
     --------------------------------------- 15.9/15.9 MB 46.8 MB/s eta 0:00:00
     ---------------------------------------- 71.9/71.9 KB 3.9 MB/s eta 0:00:00
     --------------------------------------- 46.2/46

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
%pip install jax==0.4.25 jaxlib==0.4.25 
%pip install numpy<2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


������ ������ ã�� �� �����ϴ�.


In [1]:
from mcd import sampler


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\user\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\user\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", l

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'PRNGKeyArray' from 'jax.random' (c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\jax\random.py)